In [ ]:
# Import key functions from package, also helps to have ase and other standard packages, such as numpy.
import ase
import numpy as np

from atomaton.analyze import guess_bonds
from atomaton.visualize import view_structure

In [ ]:
"""
Calculating bonds between atoms.

Many types of structure files do not have bond information, which means that these needs to be calculated to be used with certain forcefields.
To calculate bonds with atomaton, we first define a dictionary of bond lengths (in angstrom) for various atom type pairs.
Note that atom types should be listed in alphabetical order so that the default bond length is not used.
(ex. S-C will be skipped, because the code will always order the atoms types as C-S when checking the dictionary.)
"""

bond_dict = {'default':[0,2.0], 'C-C':[0, 1.8], 'C-Cl':[0, 1.8], 'C-S':[0, 1.9], 'O-S':[0, 1.9], 'H-H':[0, 0]}
ethane = ase.io.read("ethane.xyz")
irmof1 = ase.io.read("IRMOF-1.cif")

# When determining bonding, we also check for bonds which cross the unit cell, 
# and keep a list of extra atoms and extra bonds for drawing those correctly.
bonds, bond_types, bonds_across_boundary, extra_atoms, extra_bonds = guess_bonds(irmof1, cutoffs=bond_dict)


In [ ]:
# View the structure
view_structure(irmof1, bonds, bonds_across_boundary)

In [ ]:
# If you wish to view the bonds across boundaries, join the atoms with the extra atoms, and bonds with the extra bonds.
atoms_ext = irmof1+extra_atoms
bonds_ext = bonds+extra_bonds
view_structure(atoms_ext, bonds_ext, bonds_across_boundary)

In [ ]:
"""
You can view an extended cell used to calculate bonds if you create the extended cell locally.
guess_bonds actually creates a minimally extended unit cell as determined by maximum possible bond length
in O(N) time to reduce the number of atoms for the O(N^2) bond check.
"""

from atomaton.analyze import create_extended_cell_minimal 

irmof1_extended, _ = create_extended_cell_minimal(irmof1)
view_structure(irmof1_extended, [], [])

In [ ]:
"""
In order to create supercells and compound structures for simulations, you can use build functions.
One of the key ideas with these functions is that you can calculate bonds for individual structures separately, and stitch them together as needed.
This reduces the time for bond calculations, and ensures you don't accidentally create any incorrect bonds between a crystal and molecule in an unrelaxed structure.
Unlike create_extended_cell, build_supercell gives you control over how many copies to create in each direction and modifies cell parameters.
"""

from atomaton.build import insert_molecule, shift_bond_indicies

mof_bonds, mof_bond_types, mof_bonds_across_boundary, mof_extra_atoms, mof_extra_bonds = guess_bonds(irmof1, cutoffs=bond_dict)
eth_bonds, eth_bond_types, eth_bonds_across_boundary, eth_extra_atoms, eth_extra_bonds = guess_bonds(ethane, cutoffs=bond_dict)

eth_bonds_shifted = shift_bond_indicies(eth_bonds, len(irmof1))
eth_bonds_across_boundary_shifted = shift_bond_indicies(eth_bonds_across_boundary, len(irmof1))

irmof1_with_ethane, _ = insert_molecule(irmof1, ethane)
view_structure(irmof1_with_ethane, mof_bonds+eth_bonds_shifted, mof_bonds_across_boundary+eth_bonds_across_boundary_shifted)
